# EDA of the dataset

### Our datasets
main dataset  - ["European Soccer Database" (ESD)](https://www.kaggle.com/hugomathien/soccer//) 

Additional datasets -

- "UK football stadiums" -  [UK stadiums description](https://www.doogal.co.uk/FootballStadiums.php//) 
- Position in league table per stage - from [worldfootball](https://www.worldfootball.net/schedule//)

The "European Soccer Database" consists of three main tables:
- Match - features for each match of 2008-2016 seasons
- Player -  player attributes
- Team - team attributes

The worldfootball dataset was extracted directly from the website. In order to receive the position of each team in each stage of the league we wrote a script named extract_table_from_url(attached together with this file) which goes through pages such as [this](http://www.worldfootball.net/schedule/eng-premier-league-2016-2017-spieltag/9/) and write the data to an csv file. 



### EDA update 

In this notebook we worked on three issues:
1) Engineering new features 
2) Dealing with draw class poor prediction rates
3) Using Different classification models


#### New features:
- points - number of points in current league stage
- norm_points - number of points in current league stage relative to highest number of points in current stage
- position - position in league table
- straight_wins - if last game was won, number of wins in a row
- straight_loses - if last game was lost, number of losses in a row
- straight_draws - - if last game ended with a draw, number of draws in a row 
- pos_goals - goals scored in last X games
- neg_goals - goals received in last X games
- tot_pos_goals - goals scored since the begining of the season
- tot_neg_goals - goals received since the begining of the season
- goals_diff - tot_pos_goals minus tot_neg_goals
- wins - wins in last X games 
- loses - loses in last X games

X is the size of the moving window we used.
In this file we set X to be 5, in the future we will try different size for X.
All of these features were created for both home and away team for each match. 

#### Draw class:
As we saw on our first attempt classifier, the classifier never chose the draw class, to deal with this issue we tried two methods, both based on demanding a certain threshold crossing for a win/ lose decision:
1) relative threshold - the threshold is determined on the percentage win and lose cases takes out of 1.
2) absolute threshold -  win/lose classed needs to cross a predetermined threshold in order for them to be picked

#### Classification models:

In order to get a perspective of the preformance we can achive, we tried using the following models:
- Logistic Regression
- KNN
- SVM
- Decision Tree
- Random Forest
- A combination of all models

After trying all of these models the best f1-score was received using Logistic Regression with the second type of threshold.



#### General Impotrs:

In [384]:
import sqlite3
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Read all DB's:

In [385]:
#Availible Leagues:
#"England_Premier_League","France_Ligue_1","Germany_1._Bundesliga","Netherlands_Eredivisie","Spain_LIGA_BBVA"

current_leagues = ["England_Premier_League","France_Ligue_1","Germany_1._Bundesliga","Netherlands_Eredivisie","Spain_LIGA_BBVA"]

#Not Availible Leagues (yet):
#Italy_Serie_A/Switzerland_Super_League

In [386]:
leagues_db = pd.read_csv('databases/sql_leagues_db.csv', index_col = [0])
leagues_db

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [387]:
matches_db = {}

for league in current_leagues:
    matches_db[league] = pd.read_csv('databases/sql_matches_db/' + league + '.csv', index_col = [0])

matches_db[league].head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,21518,21518,21518,2008/2009,1,2008-08-30,530023,10267,8661,3,...,5.00,1.65,3.40,4.50,1.75,3.30,5.00,1.67,3.40,5.00
1,21519,21518,21518,2008/2009,1,2008-08-31,530084,8371,10205,1,...,2.60,2.90,3.20,2.25,2.75,3.20,2.50,2.80,3.25,2.30
2,21520,21518,21518,2008/2009,1,2008-08-31,530085,9783,8633,2,...,1.90,3.25,3.25,2.00,3.50,3.20,2.10,3.25,3.25,2.10
3,21521,21518,21518,2008/2009,1,2008-08-31,530086,8388,8634,1,...,1.57,7.00,3.75,1.45,6.50,3.75,1.53,5.50,3.75,1.53
4,21522,21518,21518,2008/2009,1,2008-08-31,530087,8696,8302,1,...,2.30,2.70,3.10,2.40,2.75,3.25,2.45,2.70,3.25,2.38


In [388]:
teams_db = pd.read_csv('databases/sql_teams_db.csv', encoding='latin-1', index_col = [0])
teams_db.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [389]:
league_table_db = {}

for league in current_leagues:
    league_table_db[league] = pd.read_csv('databases/table_per_stage/' + league + '.csv',encoding='latin-1', index_col = [0,1], header = [0,1], skipinitialspace = True)

league_table_db[league].head()

Stage                                  0                               \
                                     Pts Norm_Pts Position Goals_Diff   
Season    Team                                                          
2008/2009 Athletic Club de Bilbao    0.0      0.0      0.0        0.0   
          Atlético Madrid            0.0      0.0      0.0        0.0   
          CA Osasuna                 0.0      0.0      0.0        0.0   
          CD Numancia                0.0      0.0      0.0        0.0   
          RC Deportivo de La Coruña  0.0      0.0      0.0        0.0   

Stage                                                      1            \
                                    Pos_Goals Neg_Goals  Pts  Norm_Pts   
Season    Team                                                           
2008/2009 Athletic Club de Bilbao         0.0       0.0  0.0  0.000000   
          Atlético Madrid                 0.0       0.0  3.0  1.000000   
          CA Osasuna                      0.0       0.0  1.0  0.333333   
          CD Numancia                     0.0       0.0  3.0  1.000000   
          RC Deportivo de La Coruña       0.0       0.0  3.0  1.000000   

Stage                                                      ...          37  \
                                    Position Goals_Diff    ...    Position   
Season    Team                                             ...               
2008/2009 Athletic Club de Bilbao       18.0       -2.0    ...        13.0   
          Atlético Madrid                1.0        4.0    ...         4.0   
          CA Osasuna                     9.0        0.0    ...        17.0   
          CD Numancia                    6.0        1.0    ...        19.0   
          RC Deportivo de La Coruña      4.0        1.0    ...         7.0   

Stage                                                                 38  \
                                    Goals_Diff Pos_Goals Neg_Goals   Pts   
Season    Team                                                             
2008/2009 Athletic Club de Bilbao        -13.0      47.0      60.0  44.0   
          Atlético Madrid                 20.0      77.0      57.0  67.0   
          CA Osasuna                      -7.0      39.0      46.0  43.0   
          CD Numancia                    -29.0      38.0      67.0  35.0   
          RC Deportivo de La Coruña        1.0      47.0      46.0  58.0   

Stage                                                                        \
                                     Norm_Pts Position Goals_Diff Pos_Goals   
Season    Team                                                                
2008/2009 Athletic Club de Bilbao    0.505747     13.0      -15.0      47.0   
          Atlético Madrid            0.770115      4.0       23.0      80.0   
          CA Osasuna                 0.494253     14.0       -6.0      41.0   
          CD Numancia                0.402299     19.0      -31.0      38.0   
          RC Deportivo de La Coruña  0.666667      7.0        1.0      48.0   

Stage                                          
                                    Neg_Goals  
Season    Team                                 
2008/2009 Athletic Club de Bilbao        62.0  
          Atlético Madrid                57.0  
          CA Osasuna                     47.0  
          CD Numancia                    69.0  
          RC Deportivo de La Coruña      47.0  

[5 rows x 234 columns]

#### Functions:

In [390]:
def get_opposit(item):
    a = ["home","away"]
    a.remove(item)
    return a[0]

def func1(x,temp_dic,home_away,team):
    temp_dic[home_away]              = x.loc[(x[home_away + "_team_name"] == team)].copy()
    temp_dic[home_away]["home_away"] = home_away
    
def func2(x,temp_dic,home_away,features):
    temp_dic[home_away] = x.loc[(x["home_away"] == home_away)].copy()
    for item in features:
        temp_dic[home_away][home_away + "_" + item] = temp_dic[home_away][item]

def get_df(x,func,func_data):
    temp_dic = {}
    for home_away in ["home", "away"]:
        func(x,temp_dic,home_away,func_data)
    return temp_dic["home"].append(temp_dic["away"]).sort_values(["season","stage"])

def template(df,input_column,output_column,window_size):
    temp_series = df.apply(lambda x: x[x["home_away"] + "_" + input_column],axis = 1)
    df[output_column] = temp_series.rolling(window_size).sum() - temp_series
    
def opposit_template(df,input_column,output_column,window_size):
    temp_series = df.apply(lambda x: x[get_opposit(x["home_away"]) + "_" + input_column],axis = 1)
    df[output_column] = temp_series.rolling(window_size).sum() - temp_series
    
def asi(df,input_column):
    c = 0
    a = []
    for index in df.index:
        if df.at[index,df.at[index,"home_away"] + "_" + input_column] == 1: c += 1
        else: c = 0
        a.append(c)
    return ([0] + a[0:-1])

def calc_rolling_mean(df,input_column):
    series      = df[input_column]
    window_len  = len(series.index)
    temp_list   = [0] + list(series.rolling(window = window_len,min_periods = 1,center = False).mean())[0:-1]
    return pd.Series(temp_list,index = series.index)

In [391]:
import pickle

def save_obj(obj, name ):
    with open('databases/objects/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('databases/objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

#### Hyperparameters:

In [392]:
window_size = 15

#### Feature Engeneering:

In [393]:
basic_info       = ["league","season","stage","home_team_name","away_team_name","score"]
betting_odds     = ["B365H","WHH","VCH","B365A","WHA","VCA","B365D","WHD","VCD"]
last_games_wld   = [str(x)+"_last_wld" for x in list(range(1,window_size))]
new_features     = ["points","norm_points","position","straight_wins","straight_loses","straight_draws","pos_goals","neg_goals","tot_pos_goals","tot_neg_goals","goals_diff","wins","loses"] + last_games_wld
important_info   = basic_info + ["home_team_goal","away_team_goal"]

percent_per_season_columns = ["home_win_season_percent","home_lose_season_percent","away_win_season_percent","away_lose_season_percent"]
percent_total_columns      = ["home_win_percent","home_lose_percent","away_win_percent","away_lose_percent"]

home_away        = ["home","away"]

new_columns      = important_info + betting_odds

new_new_columns = []

for r in itertools.product(home_away,new_features): new_new_columns.append(r[0] +"_"+ r[1])

In [394]:
leagues_map = dict(zip(leagues_db["id"],leagues_db["name"]))
leagues_map

{1: 'Belgium Jupiler League',
 1729: 'England Premier League',
 4769: 'France Ligue 1',
 7809: 'Germany 1. Bundesliga',
 10257: 'Italy Serie A',
 13274: 'Netherlands Eredivisie',
 15722: 'Poland Ekstraklasa',
 17642: 'Portugal Liga ZON Sagres',
 19694: 'Scotland Premier League',
 21518: 'Spain LIGA BBVA',
 24558: 'Switzerland Super League'}

In [395]:
teams_map = dict(zip(teams_db["team_api_id"],teams_db["team_long_name"]))

In [396]:
all_teams   = {}
all_seasons = {}
all_stages  = {}

for league in current_leagues:
    matches_db[league]["league"]         = matches_db[league]["league_id"].map(leagues_map)
    matches_db[league]["home_team_name"] = matches_db[league]["home_team_api_id"].map(teams_map)
    matches_db[league]["away_team_name"] = matches_db[league]["away_team_api_id"].map(teams_map)
    matches_db[league]["score"] = matches_db[league]["home_team_goal"].astype("str") + " - " + matches_db[league]["away_team_goal"].astype("str")
    
    all_teams[league]   = matches_db[league]["home_team_name"].unique()
    all_seasons[league] = matches_db[league]["season"].unique()
    all_stages[league]  = matches_db[league]["stage"].unique()

    all_teams[league].sort()
    all_seasons[league].sort()
    all_stages[league].sort()

In [397]:
new_matches = {}

for league in current_leagues:
    new_matches[league] = matches_db[league][new_columns].copy()
    new_matches[league].sort_values(["season","stage"],inplace = True)

    for a in home_away:
        new_matches[league][a + "_win"]  = pd.Series(new_matches[league][a + "_team_goal"] >  new_matches[league][get_opposit(a) + "_team_goal"],dtype=int)
        new_matches[league][a + "_lose"] = pd.Series(new_matches[league][a + "_team_goal"] <  new_matches[league][get_opposit(a) + "_team_goal"],dtype=int)
        new_matches[league][a + "_draw"] = pd.Series(new_matches[league][a + "_team_goal"] == new_matches[league][get_opposit(a) + "_team_goal"],dtype=int)
        for team in all_teams[league]:
            new_matches[league].loc[new_matches[league][a + "_team_name"] == team,a + "_win_percent"]  = calc_rolling_mean(new_matches[league][new_matches[league][a + "_team_name"] == team],a + "_win")
            new_matches[league].loc[new_matches[league][a + "_team_name"] == team,a + "_lose_percent"] = calc_rolling_mean(new_matches[league][new_matches[league][a + "_team_name"] == team],a + "_lose")

    new_matches[league]["win_lose_draw"] = new_matches[league]["home_win"] + 2*new_matches[league]["away_win"]

In [398]:
new_matches[league].head()

,league,season,stage,home_team_name,away_team_name,score,home_team_goal,away_team_goal,B365H,WHH,...,home_lose,home_draw,home_win_percent,home_lose_percent,away_win,away_lose,away_draw,away_win_percent,away_lose_percent,win_lose_draw
0,Spain LIGA BBVA,2008/2009,1,Valencia CF,RCD Mallorca,3 - 0,3,0,1.7,1.67,...,0,0,0.0,0.0,0,1,0,0.0,0.0,1
1,Spain LIGA BBVA,2008/2009,1,CA Osasuna,Villarreal CF,1 - 1,1,1,2.8,2.62,...,0,1,0.0,0.0,0,0,1,0.0,0.0,0
2,Spain LIGA BBVA,2008/2009,1,RC Deportivo de La Coruña,Real Madrid CF,2 - 1,2,1,3.5,3.30,...,0,0,0.0,0.0,0,1,0,0.0,0.0,1
3,Spain LIGA BBVA,2008/2009,1,CD Numancia,FC Barcelona,1 - 0,1,0,7.0,7.00,...,0,0,0.0,0.0,0,1,0,0.0,0.0,1
4,Spain LIGA BBVA,2008/2009,1,Racing Santander,Sevilla FC,1 - 1,1,1,2.8,2.75,...,0,1,0.0,0.0,0,0,1,0.0,0.0,0


In [399]:
for league in current_leagues:
    for team in all_teams[league]:
        for season in all_seasons[league]:
            matches_per_team        = get_df(new_matches[league],func1,team)
            matches_per_team_season = matches_per_team[matches_per_team["season"] == season].copy()

            if len(matches_per_team_season.index) == 0: continue

            template(matches_per_team_season,"team_goal","pos_goals",window_size)
            opposit_template(matches_per_team_season,"team_goal","neg_goals",window_size)
            template(matches_per_team_season,"win","wins",window_size)
            template(matches_per_team_season,"lose","loses",window_size)

            temp_series = matches_per_team_season.apply(lambda x: x[x["home_away"] + "_win"],axis = 1)
            matches_per_team_season["straight_wins"]  = asi(matches_per_team_season,"win")
            matches_per_team_season["straight_loses"] = asi(matches_per_team_season,"lose")
            matches_per_team_season["straight_draws"] = asi(matches_per_team_season,"draw")

            table_per_team_season                    = league_table_db[league].transpose()[season,team]
            matches_per_team_season["points"]        = list(table_per_team_season.xs("Pts",level=1)[0:-1])
            matches_per_team_season["norm_points"]   = list(table_per_team_season.xs("Norm_Pts",level=1)[0:-1])
            matches_per_team_season["position"]      = list(table_per_team_season.xs("Position",level=1)[0:-1])
            matches_per_team_season["tot_pos_goals"] = list(table_per_team_season.xs("Pos_Goals",level=1)[0:-1])
            matches_per_team_season["tot_neg_goals"] = list(table_per_team_season.xs("Neg_Goals",level=1)[0:-1])
            matches_per_team_season["goals_diff"]    = list(table_per_team_season.xs("Goals_Diff",level=1)[0:-1])

            for a in home_away:
                matches_per_team_season.loc[matches_per_team_season[a + "_team_name"] == team,a + "_win_season_percent"]  = calc_rolling_mean(matches_per_team_season[matches_per_team_season[a + "_team_name"] == team],a + "_win")
                matches_per_team_season.loc[matches_per_team_season[a + "_team_name"] == team,a + "_lose_season_percent"] = calc_rolling_mean(matches_per_team_season[matches_per_team_season[a + "_team_name"] == team],a + "_lose")
            
            for index in matches_per_team_season.index[:window_size]:
                offset                                             = matches_per_team_season.at[index,"stage"]
                matches_per_team_season[str(offset) + "_last_wld"] = pd.Series(["NaN"]*offset + list(matches_per_team_season["win_lose_draw"])[:-offset],index=matches_per_team_season.index)

            home_away_goals = get_df(matches_per_team_season,func2,new_features)[new_new_columns]

            for index in home_away_goals.index:
                for column in new_new_columns:
                    if home_away_goals.at[index,column] != "NaN":
                        new_matches[league].at[index,column] = home_away_goals.at[index,column]

            for index in matches_per_team_season.index:
                for column in percent_per_season_columns:
                    if matches_per_team_season.at[index,column] != "NaN":
                        new_matches[league].at[index,column] = matches_per_team_season.at[index,column]

In [400]:
features_list = new_new_columns + percent_per_season_columns + percent_total_columns + betting_odds
labels_list   = "win_lose_draw"

for league in current_leagues:
    new_matches[league] = new_matches[league][basic_info + features_list + [labels_list]]

In [401]:
features_list

['home_points',
 'home_norm_points',
 'home_position',
 'home_straight_wins',
 'home_straight_loses',
 'home_straight_draws',
 'home_pos_goals',
 'home_neg_goals',
 'home_tot_pos_goals',
 'home_tot_neg_goals',
 'home_goals_diff',
 'home_wins',
 'home_loses',
 'home_1_last_wld',
 'home_2_last_wld',
 'home_3_last_wld',
 'home_4_last_wld',
 'home_5_last_wld',
 'home_6_last_wld',
 'home_7_last_wld',
 'home_8_last_wld',
 'home_9_last_wld',
 'home_10_last_wld',
 'home_11_last_wld',
 'home_12_last_wld',
 'home_13_last_wld',
 'home_14_last_wld',
 'away_points',
 'away_norm_points',
 'away_position',
 'away_straight_wins',
 'away_straight_loses',
 'away_straight_draws',
 'away_pos_goals',
 'away_neg_goals',
 'away_tot_pos_goals',
 'away_tot_neg_goals',
 'away_goals_diff',
 'away_wins',
 'away_loses',
 'away_1_last_wld',
 'away_2_last_wld',
 'away_3_last_wld',
 'away_4_last_wld',
 'away_5_last_wld',
 'away_6_last_wld',
 'away_7_last_wld',
 'away_8_last_wld',
 'away_9_last_wld',
 'away_10_last_wl

In [402]:
new_matches[league]

,league,season,stage,home_team_name,away_team_name,score,home_points,home_norm_points,home_position,home_straight_wins,...,B365H,WHH,VCH,B365A,WHA,VCA,B365D,WHD,VCD,win_lose_draw
0,Spain LIGA BBVA,2008/2009,1,Valencia CF,RCD Mallorca,3 - 0,0.0,0.000000,0.0,0.0,...,1.70,1.67,1.65,5.25,4.50,4.50,3.60,3.30,3.40,1
1,Spain LIGA BBVA,2008/2009,1,CA Osasuna,Villarreal CF,1 - 1,0.0,0.000000,0.0,0.0,...,2.80,2.62,2.90,2.50,2.30,2.25,3.30,3.20,3.20,0
2,Spain LIGA BBVA,2008/2009,1,RC Deportivo de La Coruña,Real Madrid CF,2 - 1,0.0,0.000000,0.0,0.0,...,3.50,3.30,3.25,2.10,1.95,2.00,3.30,3.25,3.25,1
3,Spain LIGA BBVA,2008/2009,1,CD Numancia,FC Barcelona,1 - 0,0.0,0.000000,0.0,0.0,...,7.00,7.00,7.00,1.50,1.40,1.45,4.00,3.75,3.75,1
4,Spain LIGA BBVA,2008/2009,1,Racing Santander,Sevilla FC,1 - 1,0.0,0.000000,0.0,0.0,...,2.80,2.75,2.70,2.50,2.25,2.40,3.30,3.20,3.10,0
5,Spain LIGA BBVA,2008/2009,1,Real Sporting de Gijón,Getafe CF,1 - 2,0.0,0.000000,0.0,0.0,...,2.62,2.40,2.50,2.62,2.60,2.65,3.30,3.10,3.10,2
6,Spain LIGA BBVA,2008/2009,1,Real Betis Balompié,RC Recreativo,0 - 1,0.0,0.000000,0.0,0.0,...,2.00,1.80,1.75,3.80,3.80,4.35,3.25,3.25,3.25,2
7,Spain LIGA BBVA,2008/2009,1,RCD Espanyol,Real Valladolid,1 - 0,0.0,0.000000,0.0,0.0,...,2.00,1.83,1.90,3.80,3.75,3.50,3.30,3.20,3.25,1
8,Spain LIGA BBVA,2008/2009,1,Athletic Club de Bilbao,UD Almería,1 - 3,0.0,0.000000,0.0,0.0,...,2.00,1.91,1.90,3.80,3.50,3.60,3.30,3.20,3.20,2
9,Spain LIGA BBVA,2008/2009,1,Atlético Madrid,Málaga CF,4 - 0,0.0,0.000000,0.0,0.0,...,1.44,1.40,1.45,7.50,7.00,7.00,4.20,3.75,3.75,1


In [403]:
final_matches_db = pd.concat((new_matches),ignore_index=True)
final_matches_db.to_csv("databases/my_matches_db.csv")

In [404]:
save_obj(window_size, "window_size" )

In [405]:
final_matches_db

,league,season,stage,home_team_name,away_team_name,score,home_points,home_norm_points,home_position,home_straight_wins,...,B365H,WHH,VCH,B365A,WHA,VCA,B365D,WHD,VCD,win_lose_draw
0,England Premier League,2008/2009,1,Manchester United,Newcastle United,1 - 1,0.0,0.000000,0.0,0.0,...,1.29,1.25,1.28,11.00,10.00,12.00,5.50,4.50,5.50,0
1,England Premier League,2008/2009,1,Arsenal,West Bromwich Albion,1 - 0,0.0,0.000000,0.0,0.0,...,1.20,1.17,1.25,15.00,12.00,13.00,6.50,5.50,6.00,1
2,England Premier League,2008/2009,1,Sunderland,Liverpool,0 - 1,0.0,0.000000,0.0,0.0,...,5.50,5.50,5.50,1.67,1.57,1.65,3.60,3.30,3.80,2
3,England Premier League,2008/2009,1,West Ham United,Wigan Athletic,2 - 1,0.0,0.000000,0.0,0.0,...,1.91,1.83,1.90,4.20,3.75,4.35,3.40,3.20,3.50,1
4,England Premier League,2008/2009,1,Aston Villa,Manchester City,4 - 2,0.0,0.000000,0.0,0.0,...,1.91,1.91,1.90,4.33,3.50,4.35,3.40,3.20,3.50,1
5,England Premier League,2008/2009,1,Everton,Blackburn Rovers,2 - 3,0.0,0.000000,0.0,0.0,...,2.00,1.95,2.05,4.00,3.50,4.00,3.30,3.10,3.30,2
6,England Premier League,2008/2009,1,Middlesbrough,Tottenham Hotspur,2 - 1,0.0,0.000000,0.0,0.0,...,3.20,2.90,3.20,2.25,2.15,2.30,3.40,3.20,3.40,1
7,England Premier League,2008/2009,1,Bolton Wanderers,Stoke City,3 - 1,0.0,0.000000,0.0,0.0,...,1.83,1.80,1.85,4.50,4.00,4.80,3.50,3.10,3.40,1
8,England Premier League,2008/2009,1,Hull City,Fulham,2 - 1,0.0,0.000000,0.0,0.0,...,2.60,2.50,2.60,2.80,2.62,2.80,3.20,2.90,3.40,1
9,England Premier League,2008/2009,1,Chelsea,Portsmouth,4 - 0,0.0,0.000000,0.0,0.0,...,1.33,1.30,1.33,10.00,8.50,11.00,5.00,4.20,5.00,1
